In [13]:
import os
from PIL import Image
from torchvision import transforms
import torch

## load custom alexnet and select layer to extract activations from

In [14]:
from models.alexnet import AlexNet
alexnet_model = AlexNet(layer_name='features.12').build()

print(alexnet_model.get_layers)

<bound method Model.get_layers of Model(
  (base_model): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dr

## load train images from tiny imagenet

In [15]:
def get_train_image_paths(train_dir):
    """ Retrieve all image paths from the training directory of Tiny ImageNet. """
    image_paths = []
    for class_dir in os.listdir(train_dir):
        class_path = os.path.join(train_dir, class_dir)
        images_path = os.path.join(class_path, 'images')
        
        if os.path.isdir(images_path):
            for image_file in os.listdir(images_path):
                image_path = os.path.join(images_path, image_file)
                if os.path.isfile(image_path):
                    image_paths.append(image_path)

    return image_paths

# Usage
train_dir = '/data/atlas/datasets/tiny-imagenet-200/train'
image_paths = get_train_image_paths(train_dir)


## define preprocess func 

In [16]:

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

# Define your transformations
preprocess = transforms.Compose([
    transforms.ToTensor(),          # Convert the image to a tensor
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)  # Normalize
])

## load and preprocess images

In [ ]:
def load_and_transform_images(image_paths, preprocess):
    """ Load images from paths and apply the specified torchvision transform. """
    return torch.stack([preprocess(Image.open(image_path).convert('RGB')) for image_path in image_paths])


processed_images = load_and_transform_images(image_paths, preprocess)
print(processed_images.shape)

## extract activations from alexnet for tiny imagenet images

In [19]:
activations = alexnet_model(processed_images)
print(activations.shape)